In [1]:
import pandas as pd
import numpy as np
from rdkit import Chem
from rdkit.Chem import AllChem

from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import PredefinedSplit, GridSearchCV

In [2]:
import sys
sys.path.append('../../../../code')

from metrics import get_hi_metrics

Skipped loading some Tensorflow models, missing a dependency. No module named 'tensorflow'
Skipped loading modules with pytorch-geometric dependency, missing a dependency. No module named 'torch_geometric'
Skipped loading modules with pytorch-geometric dependency, missing a dependency. cannot import name 'DMPNN' from 'deepchem.models.torch_models' (/home/steshin/miniconda3/envs/lohi_benchmark/lib/python3.10/site-packages/deepchem/models/torch_models/__init__.py)
Skipped loading some Jax models, missing a dependency. No module named 'jax'


In [3]:
train = pd.read_csv('../../../../data/hi/drd2/train_1.csv')
test = pd.read_csv('../../../../data/hi/drd2/test_1.csv')

train

,Unnamed: 0,smiles,value
0,383,CC(C)Oc1ccccc1N1CCN(Cc2cccc(C(=O)N3CCCCC3)c2)CC1,True
1,386,CC(C)Oc1ccccc1N1CCN(Cc2cccc(CN3CCCCC3=O)c2)CC1,True
2,389,CC(C)Oc1ccccc1N1CCN(Cc2ccccc2CN2CCCCC2=O)CC1,True
3,2695,COc1ccccc1N1CCN(CC2COCC(c3ccccc3)(c3ccccc3)O2)CC1,True
4,2995,COc1ccccc1N1CCN(C[C@H]2OCCOC2(c2ccccc2)c2ccccc...,False
...,...,...,...
2380,5444,O=C1c2ccccc2C(=O)N1CCCCN1CCCN(C(c2ccccc2)c2ccc...,True
2381,4391,O=C(CCC(=O)c1ccccc1)NCCc1c[nH]c2ccccc12,False
2382,4397,O=C(CCCC(=O)c1ccccc1)NCCc1c[nH]c2ccccc12,False
2383,5999,OC12C3C4CC5C6C4C1C6C(C53)N2CC1CCCCC1,False


# Hyperparameter Optimization

In [4]:
from scipy.spatial.distance import jaccard


def run_knn_gridsearch_tanimoto(train_fps, test_fps):
    split_index = [-1] * len(train_fps) + [0] * len(test_fps)
    pds = PredefinedSplit(test_fold = split_index)

    X = train_fps + test_fps
    y = train['value'].to_list() + test['value'].to_list()

    params = {
        'n_neighbors': [3, 5, 7, 10],
        'weights': ['uniform', 'distance'],
    }
    knn = KNeighborsClassifier(metric=jaccard)

    grid_search = GridSearchCV(knn, params, cv=pds, refit=False, scoring='average_precision', verbose=3)
    grid_search.fit(X, y)

    best_params = grid_search.best_params_
    knn = KNeighborsClassifier(n_neighbors=best_params['n_neighbors'], weights=best_params['weights'], metric=jaccard)
    knn.fit(train_fps, train['value'])

    train_preds = knn.predict_proba(train_fps)[:, 1]
    train_metrics = get_hi_metrics(train, train_preds)

    test_preds = knn.predict_proba(test_fps)[:, 1]
    test_metrics = get_hi_metrics(test, test_preds)
    return train_metrics, test_metrics


In [5]:
train_mols = [Chem.MolFromSmiles(x) for x in train['smiles']]
train_morgan_fps = [AllChem.GetMorganFingerprintAsBitVect(x, 2, 1024) for x in train_mols]

test_mols = [Chem.MolFromSmiles(x) for x in test['smiles']]
test_morgan_fps = [AllChem.GetMorganFingerprintAsBitVect(x, 2, 1024) for x in test_mols]

train_metrics, test_metrics = run_knn_gridsearch_tanimoto(train_morgan_fps, test_morgan_fps)
print(train_metrics)
print(test_metrics)

Fitting 1 folds for each of 8 candidates, totalling 8 fits
[CV 1/1] END ....n_neighbors=3, weights=uniform;, score=0.671 total time=  51.8s
[CV 1/1] END ...n_neighbors=3, weights=distance;, score=0.678 total time=  49.3s
[CV 1/1] END ....n_neighbors=5, weights=uniform;, score=0.693 total time=  48.0s
[CV 1/1] END ...n_neighbors=5, weights=distance;, score=0.704 total time=  47.3s
[CV 1/1] END ....n_neighbors=7, weights=uniform;, score=0.695 total time=  49.2s
[CV 1/1] END ...n_neighbors=7, weights=distance;, score=0.705 total time=  49.1s
[CV 1/1] END ...n_neighbors=10, weights=uniform;, score=0.693 total time=  48.0s
[CV 1/1] END ..n_neighbors=10, weights=distance;, score=0.705 total time=  47.9s
{'roc_auc': 0.9999567973814131, 'bedroc': 0.9999999999999962, 'prc_auc': 0.999970168536563}
{'roc_auc': 0.6211048815130448, 'bedroc': 0.8292465849244126, 'prc_auc': 0.7048994757526279}


# Final Evaluation

In [6]:
def fit_predict(train, test):
    train_mols = [Chem.MolFromSmiles(x) for x in train['smiles']]
    train_morgan_fps = [AllChem.GetMorganFingerprintAsBitVect(x, 2, 1024) for x in train_mols]

    test_mols = [Chem.MolFromSmiles(x) for x in test['smiles']]
    test_morgan_fps = [AllChem.GetMorganFingerprintAsBitVect(x, 2, 1024) for x in test_mols]

    knn = KNeighborsClassifier(n_neighbors=5, weights='distance', metric=jaccard)
    knn.fit(train_morgan_fps, train['value'])

    train_result = train.copy()
    train_result['preds'] = knn.predict_proba(train_morgan_fps)[:, 1]

    test_result = test.copy()
    test_result['preds'] = knn.predict_proba(test_morgan_fps)[:, 1]

    return train_result, test_result


In [7]:
for i in [1, 2, 3]:
    train = pd.read_csv(f'../../../../data/hi/drd2/train_{i}.csv')
    test = pd.read_csv(f'../../../../data/hi/drd2/test_{i}.csv')

    train_preds, test_preds = fit_predict(train, test)
    train_preds.to_csv(f'../../../../predictions/hi/drd2/knn_ecfp4/train_{i}.csv')
    test_preds.to_csv(f'../../../../predictions/hi/drd2/knn_ecfp4/test_{i}.csv')
